# TensorFlow的分类器模型构建(监督学习)归一化后--回调函数
使用数据集-> Fashion Mnist

In [1]:
import os
from tensorflow import keras
import tensorflow as tf
import sklearn
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# 这串代码让图像直接显示在Notebook里而不是单独打开一个窗口
%matplotlib inline

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

# 训练集 验证集
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

# 归一化前的max和min
print(np.max(x_train), np.min(x_train))

255 0


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

# 归一化后的max和min
print(np.max(x_train_scaled), np.min(x_train_scaled))

2.0231433 -0.8105136


In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [28,28]))
for _ in range(20):
    model.add(keras.layers.Dense(100 , activation = 'relu'))
    model.add(keras.layers.BatchNormalization()) #批归一化
    
    """
    实现批归一化的另一种方法
    model.add(keras.layers.Dense(100))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    """
    
model.add(keras.layers.Dense(10,activation = 'softmax'))

In [7]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='sgd',
    metrics=["accuracy"]
)

In [ ]:
# 记录callbacks的文件夹与文件
logdir = './callbacks_bn'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_mnist_model.h5')

#
history = model.fit(x_train_scaled, y_train,
                    epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks=[
                        keras.callbacks.EarlyStopping(patience = 5 , min_delta = 1e-3),
                        keras.callbacks.ModelCheckpoint(
                            output_model_file, save_best_only=True),
                        keras.callbacks.TensorBoard(logdir)
                    ]
                    )

W0701 00:35:40.036817 140114069792576 deprecation.py:323] From /home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 30s 539us/sample - loss: 1.0810 - accuracy: 0.6135 - val_loss: 0.6608 - val_accuracy: 0.7572
Epoch 2/20
55000/55000 [==============================] - 17s 307us/sample - loss: 0.7637 - accuracy: 0.7276 - val_loss: 0.5717 - val_accuracy: 0.7972
Epoch 3/20
55000/55000 [==============================] - 17s 308us/sample - loss: 0.6623 - accuracy: 0.7669 - val_loss: 0.5068 - val_accuracy: 0.8148
Epoch 4/20
55000/55000 [==============================] - 18s 336us/sample - loss: 0.6003 - accuracy: 0.7901 - val_loss: 0.4821 - val_accuracy: 0.8366
Epoch 5/20
55000/55000 [==============================] - 17s 310us/sample - loss: 0.5554 - accuracy: 0.8052 - val_loss: 0.4312 - val_accuracy: 0.8496
Epoch 6/20
55000/55000 [==============================] - 19s 345us/sample - loss: 0.5181 - accuracy: 0.8188 - val_loss: 0.4222 - val_accuracy: 0.8542
Epoch 7/20
55000/55000 [=====================

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()


plot_learning_curves(history)

In [ ]:
# 使用测试集验证模型的准确度
model.evaluate(x_test_scaled, y_test)